# Beta-NMF with minibatch updates for GPGPU
In this example the performance different mini-batch algorithms for Beta-NMF introduced in [serizel2016] are compared. If you need a description on how to use Beta-NMF is used to train dictionnary on a dataset X and project some test data Y on the dictionnary trained on X please refer to the [notebook related to Beta-NMF].

More detailled description of the algorithms tested below can be found in:

[serizel2016]:http://biblio.telecom-paristech.fr/cgi-bin/download.cgi?id=16155 "R. Serizel, S. Essid, and G. Richard. “Mini-batch stochastic approaches for accelerated multiplicative updates in nonnegative matrix factorisation with beta-divergence”. Accepted for publication In *Proc. of MLSP*, p. 5, 2016."
[notebook related to Beta-NMF]: https://github.com/rserizel/beta_nmf/blob/master/BetaNMF_howto.ipynb

> R. Serizel, S. Essid, and G. Richard. “Mini-batch stochastic approaches for accelerated multiplicative updates in nonnegative matrix factorisation with beta-divergence”. Accepted for publication In *Proc. of MLSP*, p. 5, 2016.

### Links
Source code is available at https://github.com/rserizel/minibatchNMF

Dcoumentation is available at http://rserizel.github.io/minibatchNMF/

## Import packages
If you plan on using GPGPU make sure you set the Theano flags correctly (see [Theano documentation]).

[Theano documentation]: http://deeplearning.net/software/theano/ 

In [13]:
from beta_nmf_minibatch import BetaNMF
from base import nnrandn
import numpy as np

The following imports are needed for plotting only. You can discard them if you do not intend to plot the results or to use another package to manage your plots.

In [14]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

## Initialiase common variables
#### Parameters
* `n_iter` : number of iterations to perform
* `n_components` : number of components for the decomposition
* `batch_size` : size of the mini-batch, depends partly on the amount of data
    * low batch size means a behaviour closer to stochastic gradient (SG)
    * high batch size mean a behaviour closer do full gradient (FG)
* `cache1_size` : size of the first cache. Adjust this variable to fit as much data as possible in the GPGPU memory and limits data transfers between central memory and GPGPU memory
* `sag_mem` : number of batches used to compute average in stochastic average gradient ([SAG]) based algorithms. SAG is a method recently introduced for optimizing cost functions that are a sum of convex functions (which is the case here). SAG provide an intermediate between FG-like methods and SG-like methods. SAG then allows to maintain convergence rate close to FG methods with a complexity comparable to SG methods.
    * `sag_mem = 0` is equivalent to SG algorithms

[SAG]: https://hal.inria.fr/hal-00860051/PDF/sag_journal.pdf "M. Schmidt, N. Le Roux, and F. Bach, “Minimizing Finite Sums with the Stochastic Average Gradient,” *ArXiv e-prints*, Sept. 2013."

#### Variables
* `X` : data to decompose
* `W_init` : initial value for W (common to all the algorithms tested below)
* `H_init` : initial value for H (common to all the algorithms tested below)

In [15]:
n_iter = 100
n_components = 10
beta = 2
batch_size = 100
cache1_size = 500
sag_mem = 2
X = nnrandn((500, 20))
H_init = nnrandn((X.shape[0], n_components))
W_init = nnrandn((X.shape[1], n_components))

## Mini-batch BetaNMF
This algorithm is equivalent to standard beta-NMF. The main advantage is that it allows to use GPGPU to decomposed large matrices that would not fit in GPGPU memory otherwise

In [16]:
nmf = BetaNMF(X.shape, n_components, beta, n_iter, verbose=10,
              cache1_size=cache1_size, batch_size=batch_size,
              W=W_init, H=H_init, init_mode='custom', solver='mu_batch')
score_mu_batch = nmf.fit(X)

Intitial score = 202196.45
Fitting NMF model with 100 iterations....
Iteration 10 / 100, duration=632.0ms, cost=1254.292308
Iteration 20 / 100, duration=1161.0ms, cost=1031.063437
Iteration 30 / 100, duration=1690.0ms, cost=942.999003
Iteration 40 / 100, duration=2126.0ms, cost=899.741224
Iteration 50 / 100, duration=2614.0ms, cost=876.343654
Iteration 60 / 100, duration=3049.0ms, cost=861.949085
Iteration 70 / 100, duration=3565.0ms, cost=852.586895
Iteration 80 / 100, duration=4000.0ms, cost=845.868132
Iteration 90 / 100, duration=4492.0ms, cost=841.049792
Iteration 100 / 100, duration=4952.0ms, cost=837.416737
Iteration 100 / 100, duration=4952.0ms, cost=837.416737


## Asymmetric SG mini-batch MU (ASG-MU) for BetaNMF
This algorithm updates `W` for each mini-batch, this approach is denoted asymmetric SG mini-batch MU (ASG-MU) as `H` and `W` are updated asymmetrically (the full `H` is updated once per epoch while `W` is updated for each mini-batch).

In [17]:
nmf = BetaNMF(X.shape, n_components, beta, n_iter, verbose=10,
              cache1_size=cache1_size, batch_size=batch_size,
              W=W_init, H=H_init, init_mode='custom', solver='asg_mu')
score_asg_mu = nmf.fit(X)

Intitial score = 202196.45
Fitting NMF model with 100 iterations....
Iteration 10 / 100, duration=590.0ms, cost=1130.336033
Iteration 20 / 100, duration=1237.0ms, cost=912.128476
Iteration 30 / 100, duration=1739.0ms, cost=867.494315
Iteration 40 / 100, duration=2220.0ms, cost=853.898667
Iteration 50 / 100, duration=2706.0ms, cost=847.255570
Iteration 60 / 100, duration=3191.0ms, cost=843.121395
Iteration 70 / 100, duration=3680.0ms, cost=840.568473
Iteration 80 / 100, duration=4188.0ms, cost=838.396760
Iteration 90 / 100, duration=4674.0ms, cost=836.763185
Iteration 100 / 100, duration=5272.0ms, cost=835.340003
Iteration 100 / 100, duration=5272.0ms, cost=835.340003


## Greedy SG mini-batch MU (GSG-MU) for BetaNMF
This algorithm updates `W` only once per epoch on a randomly selected mini-batch.

In [18]:
nmf = BetaNMF(X.shape, n_components, beta, n_iter, verbose=10,
              cache1_size=cache1_size, batch_size=batch_size,
              W=W_init, H=H_init, init_mode='custom', solver='gsg_mu')
score_gsg_mu = nmf.fit(X)

Intitial score = 202196.45
Fitting NMF model with 100 iterations....
Iteration 10 / 100, duration=703.0ms, cost=1248.259570
Iteration 20 / 100, duration=1130.0ms, cost=1029.989062
Iteration 30 / 100, duration=1556.0ms, cost=954.874274
Iteration 40 / 100, duration=1991.0ms, cost=920.177901
Iteration 50 / 100, duration=2429.0ms, cost=903.540365
Iteration 60 / 100, duration=2942.0ms, cost=894.954759
Iteration 70 / 100, duration=3373.0ms, cost=889.397084
Iteration 80 / 100, duration=3858.0ms, cost=885.006145
Iteration 90 / 100, duration=4362.0ms, cost=881.793956
Iteration 100 / 100, duration=4791.0ms, cost=879.319490
Iteration 100 / 100, duration=4791.0ms, cost=879.319490


## Asymmetric SAG mini-batch MU (ASAG-MU) for BetaNMF
This algorithm updates `W` for each mini-batch, this approach is denoted asymmetric SAG mini-batch MU (ASAG-MU) as `H` and `W` are updated asymmetrically (the full `H` is updated once per epoch while `W` is updated for each mini-batch).

In [19]:
nmf = BetaNMF(X.shape, n_components, beta, n_iter, verbose=10,
              cache1_size=cache1_size, batch_size=batch_size,
              W=W_init, H=H_init, init_mode='custom', solver='asag_mu')
score_asag_mu = nmf.fit(X)

Intitial score = 202196.45
Fitting NMF model with 100 iterations....
Iteration 10 / 100, duration=587.0ms, cost=1093.473673
Iteration 20 / 100, duration=1076.0ms, cost=888.312700
Iteration 30 / 100, duration=1581.0ms, cost=852.523551
Iteration 40 / 100, duration=2068.0ms, cost=840.599242
Iteration 50 / 100, duration=2580.0ms, cost=834.407340
Iteration 60 / 100, duration=3104.0ms, cost=830.406653
Iteration 70 / 100, duration=3595.0ms, cost=827.553189
Iteration 80 / 100, duration=4091.0ms, cost=825.480846
Iteration 90 / 100, duration=4600.0ms, cost=823.899386
Iteration 100 / 100, duration=5098.0ms, cost=822.851185
Iteration 100 / 100, duration=5098.0ms, cost=822.851185


## Greedy SAG mini-batch MU (GSAG-MU) for BetaNMF
This algorithm updates `W` only once per epoch on a randomly selected mini-batch.

In [20]:
nmf = BetaNMF(X.shape, n_components, beta, n_iter, verbose=10,
              cache1_size=cache1_size, batch_size=batch_size,
              W=W_init, H=H_init, init_mode='custom', solver='gsag_mu')
score_gsag_mu = nmf.fit(X)

Intitial score = 202196.45
Fitting NMF model with 100 iterations....
Iteration 10 / 100, duration=533.0ms, cost=1252.533378
Iteration 20 / 100, duration=967.0ms, cost=1012.371771
Iteration 30 / 100, duration=1503.0ms, cost=954.313889
Iteration 40 / 100, duration=1932.0ms, cost=936.744015
Iteration 50 / 100, duration=2359.0ms, cost=927.994787
Iteration 60 / 100, duration=2786.0ms, cost=922.665649
Iteration 70 / 100, duration=3258.0ms, cost=919.188689
Iteration 80 / 100, duration=3688.0ms, cost=917.129562
Iteration 90 / 100, duration=4131.0ms, cost=915.913167
Iteration 100 / 100, duration=4568.0ms, cost=915.031773
Iteration 100 / 100, duration=4568.0ms, cost=915.031773


## Plot the results

Create the figure:

In [21]:
output_notebook()

Loading BokehJS ...

In [22]:
p = figure(title="Mini-batch algorithms performance (cost)")

Add the data to plot:

In [23]:
#MU mini-batch
p.line(score_mu_batch[1:, 1], score_mu_batch[1:, 0], 
       legend="MU mini batch",
       line_dash=[4, 4], line_color="red", line_width=2)
#ASG-MU
p.square(score_asg_mu[1:, 1], score_asg_mu[1:, 0],
          legend="ASG-MU",
          fill_color="green", line_color="black")
p.line(score_asg_mu[1:, 1], score_asg_mu[1:, 0],
        legend="ASG-MU",
        line_color="green", line_width=2)
#GSG_MU
p.square(score_gsg_mu[2:, 1], score_gsg_mu[2:, 0],
          legend="GSG-MU",
          fill_color="green", line_color="black")
p.line(score_gsg_mu[1:, 1], score_gsg_mu[1:, 0],
        legend="GSG-MU",
        line_color="green")
#ASAG-MU
p.circle(score_asag_mu[3:, 1], score_asag_mu[3:, 0],
          legend="ASAG-MU",
          fill_color="black", line_color="black")
p.line(score_asag_mu[1:, 1], score_asag_mu[1:, 0],
        legend="ASAG-MU",
        line_color="black", line_width=2)
#GSAG_MU
p.circle(score_gsag_mu[4:, 1], score_gsag_mu[4:, 0],
          legend="GSAG-MU",
          fill_color="black", line_color="black")
p.line(score_gsag_mu[1:, 1], score_gsag_mu[1:, 0],
        legend="GSAG-MU",
        line_color="black")

Show the figure:

In [24]:
show(p)